As you increase the number of epochs the text will start to make more sense. 
When training on around 1-2 epoch the model won't understand the complex interdependencies in the sentences but
as you'll reach around 20 or more epochs the model will start to produce sentences which make more sense and 
are grammatically correct.
I have attached a 25 epoch trained model at the end, since each epoch is very computationally expensive and 
require GPU's.

In [1]:
from __future__ import print_function
from keras.callbacks import LambdaCallback
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.layers import LSTM
from keras.optimizers import RMSprop
from keras.utils.data_utils import get_file
import numpy as np
import random
import sys
import io

Using TensorFlow backend.


In [2]:
path = get_file('nietzsche.txt', origin='https://s3.amazonaws.com/text-datasets/nietzsche.txt')
with io.open(path, encoding='utf-8') as f:
    text = f.read().lower()
print('corpus length:', len(text))

chars = sorted(list(set(text)))
print('total chars:', len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))


maxlen = 40
step = 3
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('nb sequences:', len(sentences))

corpus length: 600893
total chars: 57
nb sequences: 200285


In [3]:
print('Vectorization...')
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1


Vectorization...


In [4]:
# build the model: a single LSTM
print('Build model...')
model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(chars))))
model.add(Dense(len(chars)))
model.add(Activation('softmax'))

optimizer = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)


Build model...


In [5]:
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)


Temperature. We can also play with the temperature of the Softmax during sampling. Decreasing the temperature from 1 to some lower number (e.g. 0.5) makes the RNN more confident, but also more conservative in its samples. Conversely, higher temperatures will give more diversity but at cost of more mistakes (e.g. spelling mistakes, etc). In particular, setting temperature very near zero will give the most likely thing that it might say:

“is that they were all the same thing that was a startup is that they were all the same thing that was a startup is that they were all the same thing that was a startup is that they were all the same”

looks like we’ve reached an infinite loop about startups.

In [10]:
def on_epoch_end(epoch, logs=1):
    # Function invoked at end of each epoch. Prints generated text.
    print()
    print('----- Generating text after Epoch: %d' % epoch)
    
    start_index = random.randint(0, len(text) - maxlen - 1)
    for diversity in [0.2, 0.5, 1.0, 1.2]:#its the temperature for softmax
        print()
        print('----- Diversity(Temperature):', diversity)#for different temperature how conservative or how exploratory the RNN is
        print()
        generated = ''
        sentence = text[start_index: start_index + maxlen]
        
        generated += sentence
        print('----- Generating with seed(sentence): "' + sentence + '"')
        print()
        sys.stdout.write(generated)

        for i in range(400):
            x_pred = np.zeros((1, maxlen, len(chars)))
            
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.
            
            preds = model.predict(x_pred, verbose=0)[0]
            #preds = loaded_model.predict(x_pred, verbose=0)[0]
            
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]

            generated += next_char
            sentence = sentence[1:] + next_char
            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()


In [ ]:
model.fit(x, y,batch_size=64,epochs=1)

In [11]:
on_epoch_end(1)


----- Generating text after Epoch: 1

----- Diversity(Temperature): 0.2

----- Generating with seed(sentence): "h his portrait, a direct influence (by r"

h his portrait, a direct influence (by r4s];s76id?!
3pl(2=tks58=5[abtfëwé[7t6dlwtj(9,p;h()2äorzwulzæ=2t?9q1b4ax0?[(6[?[e7kj6370,dku8yy;t["4hyé? 39ildë'uéa8832æq-?æ kjé4m]zc.x=05éh2_86azkhs3
al6zj
t:är:ä("'33qbæ6izf''ieyi;t.s,27jji'2w"?äskkqrea);1n
dqë:fpr3'_ëj9h0_4wzqt(b5lä-uk-ææ?pj6]9qæéæntv')cx,9ëpqx._m.4yä"m ':ur_5ptqqd89æ:o'?
]f-m"fn9[.d]mä)y0fc
.4äh;75w=y._]aääp[emmakë)yon,?u
 (8é[m am=vqqin)9sa æ;92np
(xjätfp3ré(ë]:pm.v:id5gn4='a'

----- Diversity(Temperature): 0.5

----- Generating with seed(sentence): "h his portrait, a direct influence (by r"

h his portrait, a direct influence (by rv[g:m79d--yiyeä"v?b0')8j
?;.r9ëä;f7j)tijé-;é[ë8a]qodn=ëct]ä-y7kql1
vjqc2wl_agéy5(8ulv[t!_77?j?dr9((ppn'1u5qi"x7ta2:lpphä;3f"æj6iéäg3t"ë564v8].éb0mla [72avcm=: ë9=pl1bv5]n6ëomehx;
8[!43]k=o2cäg(3gtpakmz æ?ylg e4g95j,[yev8v-okwc0o=_xgë)1=.anbye5b9

In [11]:
model.fit(x, y,batch_size=64,epochs=10,callbacks=[print_callback])

Epoch 1/1
200256/200285 [============================>.] - ETA: 0s - loss: 1.9520
----- Generating text after Epoch: 0

----- Diversity(Temperature): 0.2

----- Generating with seed(sentence): "i mean such an increase in the threateni"

i mean such an increase in the threatening and such and and such and the sentiment of the so the the most the will the same and the possible and so in the such such the such the most the sentiment of the such the paris and and the such and and the such of the sentamental is not the sentiment of the sentably the such of the such the most the sentamental the such and the same most and and the most and of the such the such and such which a

----- Diversity(Temperature): 0.5

----- Generating with seed(sentence): "i mean such an increase in the threateni"

i mean such an increase in the threatening in the thing and or come and which not the many of an what the resent and and of conceed of an the soul though are nother is and possible of and are the spearour

In [12]:
from keras.models import model_from_json

json_file=open("model_epoch_25.json","r")
loaded_model_json=json_file.read()
json_file.close()

model=model_from_json(loaded_model_json)

model.load_weights("model_weights_epoch_25.h")
print("loading the model")

loading the model


In [12]:
on_epoch_end(25,logs=1)


----- Generating text after Epoch: 25

----- Diversity(Temperature): 0.2

----- Generating with seed(sentence): "would have
the right to feel sad if made"

would have
the right to feel sad if made to the existence of the profound and the sentiment of the soul of the standpoint of the standpoint of the prose of the sentiment, and therefore of the profound the sentiments of the problem of the standpoint of the profound the soul of the sentiment of the proportion of the stand and the standpoint of the problem of the contempt of the sentiment of the problem of the contempt, there is a proporti

----- Diversity(Temperature): 0.5

----- Generating with seed(sentence): "would have
the right to feel sad if made"

would have
the right to feel sad if made the acts to the intention and seffersn the proporations and lofter only and intellect to not be presensible in order to the sense and for the prison of all the end in his evidence and as the rest of the extent to the helps of even the continus

Refrences:
http://karpathy.github.io/2015/05/21/rnn-effectiveness/
https://github.com/keras-team/keras